In [77]:
import re
import openpyxl
from openpyxl.utils import get_column_letter

def process_rows_to_new_excel(source_file_path, source_sheet_name, output_file_path):
    # 打开源 Excel 文件并加载指定工作表
    source_workbook = openpyxl.load_workbook(source_file_path)
    source_sheet = source_workbook[source_sheet_name]

    # 创建新的工作簿用于输出
    output_workbook = openpyxl.Workbook()

    # 创建目录工作表
    table_of_contents = "目录"
    toc_sheet = output_workbook.active
    toc_sheet.title = table_of_contents
    toc_sheet.append(["Sheet 名称", "内容摘要", "链接"])
    table_of_contents_sheet = output_workbook[table_of_contents]

    # 定义诗人列表及其对应的英文名称
    
    unmatch = []
    # 遍历源工作表的每一行
    for row_idx, row in enumerate(source_sheet.iter_rows(min_row=1, values_only=True), start=1):
        # 读取当前行的内容
        cell_value = row[0] if row[0] is not None else ""
        row_content = str(cell_value)

        # 删除方括号及其内容
        cleaned_content = re.sub(r'\[.*?\]', '', row_content)

        # 分割中文和英文内容
        chinese_text = []
        english_text = []

        # 分割句子
        sentences = re.split(r'([，。；、,.;！？!?])', cleaned_content)
        # 处理中文部分
        current_chinese_sentence = []
        poet_names = []
        for i in range(0, len(sentences), 2):
            content = sentences[i].strip()
            delimiter = sentences[i+1] if i+1 < len(sentences) else ''
            if re.search(r'[\u4e00-\u9fff]', content):
                if not poet_names:
                    for poet in poets.keys():
                        if poet in content:
                            poet_names.append(poet)
                            break
                    last_poet = max(poet_names, key=lambda p: content.rfind(p))
                    last_poet_index = content.rfind(last_poet)
                    
                    # 分割文本
                    part1 = content[:last_poet_index].strip()
                    part2 = last_poet
                    part3 = content[last_poet_index + len(last_poet):].strip()
                    current_chinese_sentence = [part1,part2,part3]
                    chinese_text.append(current_chinese_sentence)
                    current_chinese_sentence = []
                    continue
                # 判断是否包含标点符号
                if delimiter:
                    if current_chinese_sentence:
                        current_chinese_sentence.append(content)
                        chinese_text.append(''.join(current_chinese_sentence))
                        current_chinese_sentence = []
                    else:
                        chinese_text.append(content)
                else:
                    current_chinese_sentence.append(content + delimiter)

        # 添加最后一句（如果有的话）
        if current_chinese_sentence:
            chinese_text.append(''.join(current_chinese_sentence))

        # 处理英文部分
        current_english_sentence = []
        poet_names = []
        for i in range(0, len(sentences), 2):
            content = sentences[i].strip()
            delimiter = sentences[i+1] if i+1 < len(sentences) else ''
            if not re.search(r'[\u4e00-\u9fff]', content):
                if not poet_names:
                    for poet in poets.values():
                        if poet.lower() in content.lower():
                            poet_names.append(poet)
                    last_poet = max(poet_names, key=lambda p: content.rfind(p))
                    last_poet_index = content.rfind(last_poet)
                    
                    # 分割文本
                    part1 = content[:last_poet_index].strip()
                    part2 = last_poet
                    part3 = content[last_poet_index + len(last_poet):].strip()
                    current_english_sentence = [part1,part2,part3]
                    english_text.append(current_english_sentence)
                    current_english_sentence = []
                    continue
                # 判断是否包含标点符号
                if delimiter:
                    if current_english_sentence:
                        current_english_sentence.append(content)
                        english_text.append(''.join(current_english_sentence))
                        current_english_sentence = []
                    else:
                        english_text.append(content)
                else:
                    current_english_sentence.append(content + delimiter)
        # 添加最后一句（如果有的话）
        if current_english_sentence:
            english_text.append(''.join(current_english_sentence))
        chinese_text = list(filter(None,[element for item in chinese_text for element in (item if isinstance(item, list) else [item])]))
        english_text = list(filter(None,[element for item in english_text for element in (item if isinstance(item, list) else [item])]))

        
        # 校验中文和英文的行数是否一致
        if len(chinese_text) != len(english_text):
            unmatch.append(row_idx)

        # 找到第一句中文作为 sheet 名
        sheet_name = "未命名"
        if chinese_text:
            sheet_name_part = chinese_text[0].strip()
            # 替换非法字符并截断长名称
            sheet_name_part = re.sub(r'[\\/:*?"<>|[\]]', '_', sheet_name_part)
            sheet_name_part = sheet_name_part[:31]  # Excel 工作表名称最多 31 个字符
            sheet_name = sheet_name_part
        else:
            sheet_name = "未命名"

        # 创建新的工作表
        new_sheet = output_workbook.create_sheet(title=sheet_name)

        # 写入表头
        new_sheet.append(["Chinese", "English"])

        # 写入内容
        max_rows = max(len(chinese_text), len(english_text))
        for i in range(max_rows):
            chinese_sentence = chinese_text[i] if i < len(chinese_text) else ""
            english_sentence = english_text[i] if i < len(english_text) else ""
            new_sheet.append([chinese_sentence, english_sentence])

        # 设置列宽
        new_sheet.column_dimensions['A'].width = 30
        new_sheet.column_dimensions['B'].width = 30

        # 添加到目录
        link = f"#{sheet_name}!A1"
        toc_sheet.append([sheet_name, cleaned_content[:50] + '...' if len(cleaned_content) > 50 else cleaned_content, link,'unmatch' if len(chinese_text) != len(english_text) else 'match'])

    # 设置目录列宽
    toc_sheet.column_dimensions['A'].width = 30
    toc_sheet.column_dimensions['C'].width = 30

    # 设置目录链接样式
    for row in toc_sheet.iter_rows(min_row=2, max_col=3, min_col=3):
        for cell in row:
            cell.hyperlink = cell.value
            cell.style = "Hyperlink"
    print(unmatch)
    # 保存输出工作簿
    output_workbook.save(output_file_path)
    output_workbook.close()
    source_workbook.close()

poets = {
        "李白": "Li Bai",
        "王维": "Wang Wei",
        "孟浩然": "Meng Haoran",
        "李商隐": "Li Shangyin",
        "杜甫": "Du Fu",
        "白居易": "Bai Juyi",
        "柳宗元": "Liu Zongyuan",
        "杜牧": "Du Mu",
        "陈子昂": "Chen Zi'ang",
        "王昌龄": "Wang Changling",
        "高适": "Gao Shi",
        "元稹": "Yuan Zhen",
        "岑参": "Cen Shen",
        "韩愈": "Han Yu",
        "刘禹锡": "Liu Yuxi",
        "韦应物": "Wei Yingwu",
        "贾岛": "Jia Dao",
        "张继": "Zhang Ji",
        "李颀": "Li Qi",
        "常建": "Chang Jian",
        "戴叔伦": "Dai Shulun",
        "顾况": "Gu Kuang",
        "张籍": "Zhang Ji",
        "王建": "Wang Jian",
        "张祜": "Zhang Hu",
        "司空曙": "Sikong Shu",
        "李端": "Li Duan",
        "卢纶": "Lu Lun",
        "孟郊": "Meng Jiao",
        "鲍溶": "Bao Rong",
        "雍陶": "Yong Tao",
        "刘长卿": "Liu Changqing",
        "郎士元": "Lang Shiyuan",
        "钱起": "Qian Qi",
        "杨巨源": "Yang Juyuan",
        "卢汝弼": "Lu Rushi",
        "吴融": "Wu Rong",
        "韩偓": "Han Wo",
        "李频": "Li Pin",
        "林宽": "Lin Kuan",
        "殷文圭": "Yin Wengui",
        "黄滔": "Huang Tao",
        "翁洮": "Weng Tao",
        "齐己": "Qi Ji",
        "贯休": "Guan Xiu",
        "昙域": "Tan Yu",
        "修睦": "Xiu Mu",
        "张乔": "Zhang Qiao",
        "李中": "Li Zhong",
        "李洞": "Li Dong",
        "处默": "Chu Mo",
        "骆宾王": "Luo Binwang",
        "贺知章": "He Zhizhang",
        "黄巢": "Huang Chao",
        "王勃": "Wang Bo",
        "李益": "Li Yi",
        "崔护":"Cui Hu",
        "徐凝":"Xu Ning",
        "崔颢":"Cui Hao",
        "皮日休":"Pi Rixiu",
        "杨炯":"Yang Jiong",
        "王之涣":"Wang Zhihuan",
        "李贺":"Li He",
        "李绅":"Li Shen",
        "无名氏":"Anonymous"
    }

# 使用示例
source_file_path = "1.xlsx"  # 替换为你的源文件路径
source_sheet_name = "Sheet1"  # 替换为你的源数据表名称
output_file_path = "output.xlsx"  # 替换为你想要的输出文件路径
process_rows_to_new_excel(source_file_path, source_sheet_name, output_file_path)

[1, 7, 8, 9, 10, 12, 15, 17, 18, 19, 21, 23, 24, 25, 27, 29, 30, 31, 34, 37, 38, 39, 40, 41, 42, 45, 46, 47, 48, 49, 50, 52, 53, 57, 58, 60, 61, 64, 65, 66, 68, 69, 70, 71, 73, 74, 76, 77, 79, 80, 81, 83, 84, 85, 86, 88, 90, 91, 92, 93, 94, 95, 96, 100]
